In [149]:
import h5py as h5
import numpy as np
import os
import pandas as pd
from scipy.optimize import curve_fit

In [163]:
Q_c = 10600
phi0 = 0.02

def read_hdf5(dir = 'F:\\LabIV\\QTLab2324\\IRdetection\\Instruments\\Test_data\\data_19_02_set1\\'):

    dfs, temp = [], []

    for T_titles in os.listdir(dir):
        subdir = dir + T_titles
        temporary = []
        for file in os.listdir(subdir):
            with h5.File(dir+T_titles+'\\'+file, 'r') as file:
                group = file.require_group('raw_data')
                df = pd.DataFrame()
                for (i,key) in enumerate(group.keys()):
                    df.insert(i, column=key, value=group[key][:])
                df['S21'] = 20*np.log10(np.sqrt(np.array(df['i'])**2 + np.array(df['q'])**2))
                temporary.append(df)
        temp.append(int(T_titles.replace('T_','')))
        dfs.append(temporary)
    
    return dfs, temp  

def FWHM(x, y):
    x=np.array(x)
    y=np.array(y)
    # Find the maximum value of y
    max_index = np.argmax(y)
    max_y = y[max_index]

    # Find the indices where y is half of the maximum
    half_max = max_y / 2.0
    left_index = np.argmin(np.abs(y[:max_index] - half_max))
    right_index = np.argmin(np.abs(y[max_index:] - half_max)) + max_index

    # Interpolate to find the exact x values corresponding to the half-maximum points
    left_x = np.interp(half_max, [y[left_index - 1], y[left_index]], [x[left_index - 1], x[left_index]])
    right_x = np.interp(half_max, [y[right_index], y[right_index + 1]], [x[right_index], x[right_index + 1]])

    # Compute FWHM
    fwhm = right_x - left_x
    return fwhm

def global_minimum_index(data):
    data = np.array(data)
    """Find the index where the function reaches its global minimum."""
    min_index = 0
    min_value = data[0]  # Initialize with the first element

    for i, value in enumerate(data):
        if value < min_value:
            min_value = value
            min_index = i

    return min_index

def estimate_bg_coeff(x,y, degree=1):
    return np.polyfit(x, y, degree)

def Q_stima(x0,w):
    if (((x0 is None) or (w is None)) or w==0):
        print('Q_stima: None detected!')
        return 0
    else:
        return x0/(2*w)
    
def resonance(x, x0, Q):
    num = np.exp(1j*phi0)
    den = 1 + 2*1j*Q*(x-x0)/x0
    return abs(1 - (Q/Q_c)*(num/den))

def model(x,coeff, x0,Q):
    return np.polyval(coeff, x) * resonance(x,x0,Q)

In [159]:
dfs, temps = read_hdf5()

In [164]:
pars = []

for df_t in dfs:
    for df_f in df_t:
        width = FWHM(df_f['f'],df_f['S21'])
        x0 = df_f['f'][global_minimum_index(df_f['f'])]
        Q = Q_stima(x0,width)
        coeff = estimate_bg_coeff(df_f['f'],df_f['S21'],degree=1)
        Q_c = 10600
        phi = 0.2
        pars.append([coeff,x0, Q, Q_c, phi])

In [165]:
popts = []
for (i,df_t) in enumerate(dfs):
    for (j,df_f) in enumerate(df_t):
        try:
            popt, pcov = curve_fit(model, df_f['f'], df_f['S21'], p0=pars[i*4+j], maxfev=10000)
            popts.append(popt,pcov)
        except:
            print(i*4+j)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
